# Imports

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [1]:
from workflow_widgets import WorkflowWidget, WorkerPoolWidget, TailWidget
from droplet_workflow import droplet_wf
from example_workflow import example_wf
from aux_widgets import QueueWidget, NEWTAuthWidget, SSHAuthWidget, SSHTerminal, Space
import workflow_objects as kale

import ipywidgets as ipw
import time
from concurrent.futures import ThreadPoolExecutor
import traitlets
import networkx as nx
import IPython.display as disp

# Define simple workflow

In [2]:
wpw = WorkerPoolWidget()
wpw

A Jupyter Widget

In [4]:
wfw = WorkflowWidget(example_wf, wpw)
wfw

A Jupyter Widget

In [ ]:
example_workflow = kale.Workflow(name='example')
for i in range(10):
    example_workflow.add_task(
        kale.CommandLineTask(
            name='echo_test_{num}',
            command='echo "Test #{num}" >> {out_file} &&',
            tags=['run'],
            output_files=["{out_file}"],
            params=dict(
                num=i,
                out_file="output.txt"
            )
        )
    )

In [5]:
wfw.workflow.tag_dict

{'done': [<workflow_objects.CommandLineTask at 0x7effcccdc7f0>],
 'echo': [<workflow_objects.CommandLineTask at 0x7effcccdc6a0>,
 'pwd': [<workflow_objects.CommandLineTask at 0x7effcccdc668>]}

In [ ]:
wfw.workflow._new_tag('newtag')

In [ ]:
wfw._update_tag_selector()

# Connect to Cluster


In [ ]:
ssh = SSHAuthWidget(host='cori.nersc.gov', username='oevans')
ssh

In [ ]:
t = SSHTerminal(ssh, width=800)
t

# Create Worker Pool


# Visualize & Run Workflow

In [6]:
ww = WorkflowWidget(workflow=droplet_wf, worker_pool_widget=wpw)
ww

A Jupyter Widget

In [ ]:
exww.workflow.tag_dict

In [ ]:
selected = exww.bqgraph.selected
tasks = [exww.bqgraph.node_data[x]['name'] for x in selected]
[exww.workflow.get_task_by_name(name) for name in tasks]

In [ ]:
nx.dfs_preorder_nodes?

In [ ]:
exww.workflow.dag.o(node)

In [ ]:
exww.bqgraph.selected = list(range(5))

In [ ]:
example_wf.dag.nodes()[1].children

# Observe Output

In [ ]:
t = TailWidget('out.txt')
t

# Connect to NEWT

In [ ]:
a = NEWTAuthWidget()
a

In [ ]:
q = QueueWidget(a)
q